In [1]:
# Import required modules 
import sqlite3
import pandas as pd
import numpy as np

In [2]:
# Connect to SQLite database
conn = sqlite3.connect("../sql/synthea.db")  # SQL used for showmanship, because the analysis can be done without it

In [87]:
# Read csv files to database

# First, save them as pandas df
patients = pd.read_csv('../csv/patients.csv')
encounters = pd.read_csv('../csv/encounters.csv')
medications = pd.read_csv('../csv/medications.csv')
conditions = pd.read_csv('../csv/conditions.csv')

# Second, load tables into database
patients.to_sql('patients', conn, if_exists='replace', index=False)
encounters.to_sql('encounters', conn, if_exists='replace', index=False)
medications.to_sql('medications', conn, if_exists='replace', index=False)
conditions.to_sql('conditions', conn, if_exists='replace', index=False)


212079

In [88]:
# Load tables 
patients = pd.read_sql_query('SELECT * FROM patients', conn)
encounters = pd.read_sql_query('SELECT * FROM encounters', conn)
medications = pd.read_sql_query('SELECT * FROM medications', conn)
conditions = pd.read_sql_query('SELECT * FROM conditions', conn)


In [89]:
# Filter inpatient encounters
inpatients = encounters[encounters['ENCOUNTERCLASS'] == 'inpatient'].copy()

print(f'Total inpatient encounters: {len(inpatients)}')

Total inpatient encounters: 6031


In [90]:
# Create common antibiotic keywords
antibiotic_keywords = [
    'cillin', 'mycin', 'cef', 'ceph', 'oxacin', 'penem', 'azole', 
    'cycline', 'doxy', 'clinda', 'quinolone'
    ]

# Define function for checking if a medication is an antibiotic
def is_antibiotic (name):
    if pd.isna(name):
        return False
    name_lower = name.lower()
    return any(keyword in name_lower for keyword in antibiotic_keywords)

# Get medications data of patients who received antibiotics
medications['is_antibiotic'] = medications['DESCRIPTION'].apply(is_antibiotic)
antibiotics = medications[medications['is_antibiotic']].copy()


In [91]:
# Merge with inpatient encounters
abx_inpatient = pd.merge(
    left=antibiotics,
    right=inpatients,
    on='PATIENT',
    suffixes=['_med', '_enc']
)

# Merge with patient demographics
abx_inpatient = pd.merge(
    left=abx_inpatient,
    right=patients[['Id', 'GENDER', 'BIRTHDATE']],
    left_on='PATIENT',
    right_on='Id',
    suffixes=['_pt', '_medenc'],
    how='left'
)

In [92]:
# Compute temporal windows

# Create function to ensure all dates are TZ-naive
def to_naive_datetime(date_series):
    return pd.to_datetime(date_series, utc=True).dt.tz_localize(None)

# Convert all dates
for col in ['START_med', 'STOP_med', 'START_enc', 'STOP_enc', 'BIRTHDATE']:
    abx_inpatient[col] = to_naive_datetime(abx_inpatient[col])

# Keep only meds that overlap admission
# Allow 2 days pre-admission for ED therapy
pre_admission_window = pd.Timedelta(days=2)
abx_inpatient = abx_inpatient[
    (abx_inpatient['STOP_med'] >= abx_inpatient['START_enc'] - pre_admission_window) &
    (abx_inpatient['START_med'] <= abx_inpatient['STOP_enc'])
]

# Compute:
# - Antibiotic start relative to start of admission
abx_inpatient['days_from_admission_start'] = (abx_inpatient['START_med'] - abx_inpatient['START_enc']).dt.days
# Drop extreme negatives
abx_inpatient = abx_inpatient[abx_inpatient['days_from_admission_start'] >= -2] # 567 rows dropped to 556

# - Antibiotic duration
abx_inpatient['abx_duration_days'] = (abx_inpatient['STOP_med'] - abx_inpatient['START_med']).dt.days + 1 # to include same-day antibiotic regimen

# - Age at admission
abx_inpatient['age_at_admission'] = (abx_inpatient['START_enc'] - abx_inpatient['BIRTHDATE']).dt.days // 365



In [93]:
# Cumulative exposure per admission

admission_level = (
    abx_inpatient
    .groupby(['PATIENT', 'ENCOUNTER'])
    .agg(
        total_abx_days=('abx_duration_days', 'sum'),
        num_abx_courses=('abx_duration_days', 'count'),
        first_abx_day=('days_from_admission_start', 'min')
    )
    .reset_index()
)

In [94]:
# Cumulative exposure per patient

patient_level = (
    admission_level
    .groupby('PATIENT')
    .agg(
        cumulative_abx_days=('total_abx_days', 'sum'), # sum across admissions
        total_abx_courses=('num_abx_courses', 'sum'), # sum courses
        admissions_with_abx=('ENCOUNTER', 'nunique'), # count admissions
        earliest_abx_day=('first_abx_day', 'min') # first start day ever
    )
    .reset_index()
)

In [95]:
# Categorize antibiotic exposure 

admission_level['exposure_category'] = pd.cut(
    admission_level['total_abx_days'],
    bins=[-1, 0, 5, 14, 30, float('inf')],
    labels=['None', 'Short (1-5)', 'Medium (6-14)', 'High (15-30)', 'Very High (>30)']
)

patient_level['exposure_category'] = pd.cut(
    patient_level['cumulative_abx_days'],
    bins=[-1, 0, 5, 14, 30, float('inf')],
    labels=['None', 'Short (1-5)', 'Medium (6-14)', 'High (15-30)', 'Very High (>30)']
)

In [96]:
# Get antibiotic-resistant infections

# Filter for antibiotic-resistant infections
resistant_keywords = ['mrsa', 'vre', 'esbl']

conditions['is_resistant'] = conditions['DESCRIPTION'].str.lower().str.contains(
    '|'.join(resistant_keywords ) + '|resistant|resistance'
)

resistant_conditions = conditions[conditions['is_resistant']].copy()

# resistant_conditions.head() # Empty dataframe, so no antibiotic-resistant condition in synthetic dataset.

In [97]:
# Simulate resistance events (since conditions dataset has no resistant conditions)

np.random.seed(5) # for reproducibility

# Probability of resistance capped at 1
admission_level['resistance_event'] = np.random.binomial(
    n=1,
    p=np.minimum(admission_level['total_abx_days']/14, 1) # more antibiotic days = higher probability
)

In [ ]:
# Simulate day of resistance

def simulate_resistance_day (row):
    if row['resistance_event'] == 1:
        return np.random.randint(0, int(row['total_abx_days'])+1) # random day from start of admission to end of abx
    else:
        return np.nan

admission_level['resistance_day'] = admission_level.apply(simulate_resistance_day, axis=1)

,PATIENT,ENCOUNTER,total_abx_days,num_abx_courses,first_abx_day,exposure_category,resistance_event,resistance_day
0,00d07349-66d6-7a87-4238-03f4d3ebefc8,00d07349-66d6-7a87-69cf-59fd1b1de554,1,1,-1,Short (1-5),0,NaN
1,013791bd-a9fa-7887-dd8b-fc8e4a2ea985,013791bd-a9fa-7887-86d9-b8cddc1553f4,3,3,-2,Short (1-5),1,1.0
2,0191052f-e53b-d0c4-c11d-b0261d88c775,0191052f-e53b-d0c4-dd65-d1882e9dd350,16,16,0,High (15-30),1,14.0
3,0239bfce-6a07-7072-61d0-bc8eac1033bb,0239bfce-6a07-7072-d0a3-484c381bfbb1,2,2,-1,Short (1-5),1,2.0
4,030116f2-020a-f342-bed8-8480c456ad6b,030116f2-020a-f342-2bf5-b506d4933b13,1,1,-1,Short (1-5),0,NaN


In [99]:
# Save final temporal datasets
admission_level.to_csv('../csv/admission_level_with_resistance.csv')
patient_level.to_csv('../csv/patient_level_cumulative_exposure.csv')